In [7]:
import numpy as np

In [6]:
print("Ejercicio 1")

p = "inf" # 0,1,2,3,4,5,...,"inf"

x = np.random.random((3,3))
print("X: " + str(x))

if p == 0:
    p_norms = np.sum(x > 0)
elif p == "inf":
    p_norms = np.max(x, axis=1)
else:
    p_norms = np.sum(np.abs(x)**p, axis=1)**(1/p)

print("l-" + str(p) + " norm: " + str(p_norms))

Ejercicio 1
X: [[0.99481096 0.43481343 0.03046574]
 [0.68265141 0.72233814 0.48683513]
 [0.0498044  0.92298952 0.08560792]]
l-inf norm: [0.99481096 0.72233814 0.92298952]


In [9]:
print("Ejercicio 2")

x = np.random.random((3,3))
print("X: " + str(x))

p = 2
p_norms = np.sum(np.abs(x)**p, axis=1)**(1/p)

indexes = np.argsort(p_norms * -1)
print("Indexes: " + str(indexes))
print(x[indexes])

Ejercicio 2
X: [[0.52185089 0.06188702 0.82164098]
 [0.35522867 0.21280792 0.30016442]
 [0.80058939 0.70260974 0.42875391]]
Indexes: [2 0 1]
[[0.80058939 0.70260974 0.42875391]
 [0.52185089 0.06188702 0.82164098]
 [0.35522867 0.21280792 0.30016442]]


In [16]:
print("Ejercicio 3")

class IdxManager:
    def __init__(self, ids):
        self.idx2id = ids
        self.id2idx = []

        # remove duplicated values in ids
        indexes = np.unique(ids, return_index=True)[1]
        ids = np.array([ids[index] for index in sorted(indexes)])

        self.id2idx = np.full((np.max(ids) + 1),-1)
        self.id2idx[ids] = np.arange(ids.size)

    def get_users_idx(self, user_id):
        try:
            return self.id2idx[user_id]
        except IndexError:
            return -1

    def get_users_id(self, idx):
        try:
            return self.idx2id[idx]
        except IndexError:
            return -1

idxManager = IdxManager([15, 12, 14, 10, 1, 2, 1])

print(idxManager.get_users_idx(15))
print(idxManager.get_users_idx(12))
print(idxManager.get_users_idx(3))

print(idxManager.get_users_id(0))
print(idxManager.get_users_id(4))

Ejercicio 3
0
1
-1
15
1


In [17]:
print("Ejercicio 4")

truth       = np.array([1,1,0,1,1,1,0,0,0,1])
prediction  = np.array([1,1,1,1,0,0,1,1,0,0])

TP = np.sum(truth & prediction)
TN = np.sum((truth == False) & (prediction == False))
FN = np.sum((truth == True) & (prediction == False))
FP = np.sum((truth == False) & (prediction == True))

precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)

print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("Accuracy: " + str(accuracy))

Ejercicio 4
Precision: 0.5
Recall: 0.5
Accuracy: 0.4


In [18]:
print("Ejercicio 5")

T = True
F = False

q_id            = np.array([1,1,1,1,2,2,2,3,3,3,3,3,4,4,4,4])
predicted_rank  = np.array([0,1,2,3,0,1,2,0,1,2,3,4,0,1,2,3]) # Para qué me sirve ?
truth_relevance = np.array([T,F,T,F,T,T,T,F,F,F,F,F,T,F,F,T])

q_precision = np.zeros((4,1))

q_precision[0] = truth_relevance[q_id == 1].mean()
q_precision[1] = truth_relevance[q_id == 2].mean()
q_precision[2] = truth_relevance[q_id == 3].mean()
q_precision[3] = truth_relevance[q_id == 4].mean()

for i in range(4):
    print("Precision for id " + str(i) + ": " + str(q_precision[i]))

print("Average query precision: " + str(q_precision.mean()))

Ejercicio 10
Precision for id 0: [0.5]
Precision for id 1: [1.]
Precision for id 2: [0.]
Precision for id 3: [0.5]
Average query precision: 0.5


In [10]:
print("Ejercicio 10")

import pickle
import csv

class MovieRatings:
    
    instance = None
    data = None
    
    def __new__(cls, fname):
        if MovieRatings.instance is None:
            print("Creating new MovieRatings instance")
            MovieRatings.instance = super(MovieRatings, cls).__new__(cls)
            return MovieRatings.instance
        else:
            return MovieRatings.instance
        
    def __init__(self, fname):
        print("Initialising MovieRatings")
        
        try:
            with open(fname + '.pkl','rb') as pkl_file:
                self.data = pickle.load(pkl_file)
        except FileNotFoundError:
            print("CSV file found. Building PKL file...")
            try:
                with open(fname + '.csv') as csv_file:
                    with open(fname + '.pkl','wb') as pkl_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        first_skipped = False
                        a = [] # Aparentemente no puedo crear un numpy array dinámico para ir guardando
                        for line in csv_reader:
                            if not first_skipped:
                                first_skipped = True
                                continue
                            a.append((line[0],line[1],line[2],line[3]))
                        structure = [('userId', np.int32),
                                    ('movieId', np.int32),
                                    ('rating', np.float32),
                                    ('timestamp', np.int64)]
                        array = np.array(a, dtype=structure)
                        pickle.dump(array, pkl_file, protocol=pickle.HIGHEST_PROTOCOL)
                        
                    pkl_file.close()
                
                with open(fname + '.pkl','rb') as pkl_file:
                    self.data = pickle.load(pkl_file)
            except FileNotFoundError:
                print("No PKL or CSV named " + fname + " was found.")
            finally:
                csv_file.close()
        finally:
            pkl_file.close()

obj = MovieRatings('ratings_test')
print(obj.data[1])

obj2 = MovieRatings('ratings_test')

Ejercicio 10
Creating new MovieRatings instance
Initialising MovieRatings
(1, 147, 4.5, 1425942435)
Initialising MovieRatings
